# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [54]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [73]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [76]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 翻轉股災宿命！新興金融科技FinTech客製化穩健投資成黑馬

▲受疫情影響全球股匯震盪，許多投資人淪為受災戶。(示意圖／取自圖庫Pexels)
財經中心／綜合報導
新冠肺炎(COVID-19)疫情狂釀股匯震盪，在國際金融市場掀起滔天大浪，就連傳統避險標的黃金與比特幣也應聲而倒，甚至股神巴菲特都深深感嘆活了89年來第一次目睹美股熔斷。然而受黑天鵝行情重擊，國際金融市場一片哀鴻遍野之際，卻有數家金融科技（FinTech）平台業者靠著其AI智能程序化自動交易模組讓投資人成功避險且持續穩健獲利，吸引全球投資人目光。
▲「德萊富Dollars Life資產管理集團」CEO Victor Fan。（圖／德萊富提供）
近年一波波互聯網的應用浪潮，為FinTech時代推波助瀾，陸續有傳統金融業務項目轉換成為結合雲端智能的新形態金融科技商品，其中AI智能程序化自動高頻交易模組更引人注目。深耕金融產業已逾20年的Victor Fan表示，有感於整體投資環境對一般投資人長期存在不公平、不友善的不對等關係，加上一般投資人總難以熟知善用各式新投資科技工具，最嚴重的是存在著國際間的高端理財資訊落差，導致多數投資人最後只能重複上演淪為各式金融遊戲之下的白老鼠！因此，為了降低投資人的投資風險並希望借重互聯網科技力量給予投資人更友善的投資環境與信心，Victor Fan毅然創立「德萊富Dollars Life資產管理集團」。
▲國際間高端理財落差，使許多投資人淪為金融遊戲下的白老鼠。(示意圖／取自Pexels)
德萊富資產管理集團長期聘請與培訓的資深國際財經產業分析師團隊，透過不間斷地觀察預測分析國際金融情勢後，定期彙整成雲端大數據系統資料庫，再交由分布在新加坡、瑞士、上海、深圳、香港與台北的跨國金融技術團隊經過反覆地回測、預測開發出AI自動化智能化高頻金融理財平台，並導入區塊鏈去中心化應用技術，從技術面、資安面、趨勢面以及AI自動化決策交易系統等多方機制，終於在2018年底推出首套AI智能程序化自動交易高頻模組系統，該系統主要核心基礎為優先保護投資成本，進而再講求穩健的自動化獲利交易運作，集團並於2019年第三季推出第二代超進化的類量子計算AI智

▲經濟部長沈榮津。（圖／記者李毓康攝） 
記者林銘翰／台北報導
新冠肺炎疫情延燒，國民黨立委溫玉霞25日在立法院質詢時表示，中國大陸在疫情的壓力之下，是否有可能中止兩岸經濟合作架構協議（ECFA），國安局長邱國回應的確有這個可能。對此，經濟部長沈榮津27日表示，目前還沒有收到相關通知，若收到中止的通知也是在半年後生效，如果有影響就是在5%的範圍內。
溫玉霞25日在立法院質詢時表示，新冠肺炎疫情衝擊，導致中國大陸的習政權「有點困難」，詢問中國大陸是否可能在疫情壓力下，中止與台灣的ECFA，轉移內部壓力。邱國正回應，的確有這個可能，疫情的紛紛擾擾讓中國大陸需要找一個出口點，國安局目前仍在注意中。
行政院長蘇貞昌27日率領行政團隊赴立法院進行施政報告備詢，沈榮津會前受訪時表示，目前還沒有收到相關通知，若收到中止的通知也是在半年後生效，如果有影響就是在5%的範圍內。
沈榮津說，目前最重要的是產業要升級轉型，以及要多元佈局，這才是當前的因應之道。
此外，沈榮津日前於ICT產業座談會後指出，許多店家已經等不及疫情過後的振興抵用券，政府因此將推出疫情期間就可以使用的「酷碰券」，只要綁定街口、台灣Pay等數位行動支付，出門實體消費，就提供20%至25%的折扣，回饋給消費者，刺激內需。
對此，沈榮津表示，現在大家期待在疫情期間應該啟動折扣刺激消費，現在首先要防止群聚消費，將其改為日常消費，但最重要的是要先有消費才能折扣，才不會產生替代消費。經濟部若有新政策要推出，都會事先拍好宣導影片，屆時也歡迎大家到經濟部的網站觀看。
其他人也看了這些...
更多熱門新聞...
----------------------------------------------------------------------

[9] 觀光業員工薪資補助加倍　台灣大車隊董座：盼計程車司機也納入

▲台灣大車隊七度攜手董氏基金舉辦戒菸公益活動，董事長林村田也向政府喊話，希望政府能將計程車司機納入觀光紓困補助名單內。（圖／台灣大車隊提供）
記者林淑慧／台北報導
行政院昨（26）日宣佈加碼觀光旅宿業薪資補助方案，台灣大車隊董事長林村田表示，新冠肺炎（COVID-19）疫情延燒，重擊國內旅運市場，推估計程車路邊攔車業績年減至少五成，衝擊全台9.2萬名司機的生計，期盼政府也能比照觀光從業人員予以薪資補助。
新冠肺炎疫

記者鄒鎮宇／綜合報導
美國26日晚間公布請領失業補助金的人數，原超過預期的200萬人，人數來到328.3萬，相當於美國就業人口2%。對此，美股開盤暴漲500點或2.35%，航空股走高，達美航空漲3.8%、美國航空集團漲4.1%、聯合大陸航空漲4%。
道瓊工業指數開盤飆漲500點或2.35％，報21652.87點大關；標普500指數漲39.24點或1.59%，報2514.8點；納斯達克指數漲107.621點或1.46%，報7490.440點。
受到疫情影響，實體經濟快速惡化，美國不少中小企業、店面關閉，進而導致失業率飆升，即使美國提出不少策略，仍無法搶救經濟的惡化。
美國經濟諮商局研究顯示，接下來幾周，預估會有2300萬人失業，佔美國總人口數的15%。美聯儲官員不拉德指出，失業率可能會飆漲到30%，比經濟大蕭條還要高。
另外，聯準會 (Fed) 主席鮑爾26日接受《NBC》採訪，坦承新冠肺炎確實為美國的經濟帶來龐大的壓力，但他們降息注入數兆美元，預期可帶動市場的流動性，未來經濟反彈的情況會相對良好。
【23：44更新】
道瓊上漲1208點或5.71%，報22363.46點；標普500漲116.22點或4.69%，報2591點；納斯達克漲284.03點或3.85%，報7666.85點。
►酒精乾洗手特惠組！～防疫殺菌神器來囉～
----------------------------------------------------------------------

[22] 一家6口曾擠8坪房舍！財金教授智商160　靠法拍幫家人脫離無殼命運

鏡週刊
智力測驗160分，從小擁有過人的數學與空間能力，聶建中不僅是大學財金系教授，更是擁有多間房產的包租公。聶建中之所以鍾情投資房地產，跟他童年的記憶有關，1家6口只能擠在沒有隔間的8坪小屋，讓他立志為家人買房圓夢。
 
好奇赴美攻讀擁有一博四碩高學歷的聶建中，怎麼會接觸法拍？「小時候很窮，住在台中的空軍眷村，但父親是綠徽士官，1家6口只能擠在沒有隔間的8坪房舍。」買東西講求經濟實惠的聶建中，發現法拍屋更加「俗擱大碗」，從此開始研究，並在1997年在台中買下第一間法拍屋，為家人圓夢。
「沒想到原屋主竟找上門希望將房子買回去。」看在對方誠懇且遭逢人生困境，聶建中同意賣出，因此在短短2個月內獲利44萬元，讓他見識到法拍屋的魅力。

▲ 台股劇烈波動，造成不少企業股價下挫。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
近期股市大幅波動，衝擊不少企業股價重挫，金管會也鼓勵企業實施庫藏股，但有部分企業受限可轉換公司債掛牌三個月內不得實施庫藏股的規定，而向金管會反映希望放寬規範，證期局表示，該項規範是為保護股東權益，是否有條件放寬，也會與券商公會研議。
中華民國證券商業同業公會《承銷商會員輔導發行公司募集與發行有價證券自律規則》第四條之十二規定，承銷商輔導發行公司或外國發行人辦理申報募資案件時，應輔導發行公司或外國發行人於申報案件時出具承諾書，承諾自申報日起至該有價證券掛牌後三個月內不得買回本公司股份。
證期局官員指出，此項規範目的在於防止投資人短期套利損及股東權益。若發行公司募資可轉換公司債之後，實施庫藏股提高股價，在放空之後再去執行公司債拿回股票，這種短期套利行為會嚴重影響股東權益，105年時有發生過公司藉此套利的情形，因此106年1月1日即規範企業在可轉換公司債掛牌的三個月內，禁止實施庫藏股。
證期局官員表示，有個別公司通過民意代表向證期局反映，本次受到股市重挫影響，有部分股東希望進行庫藏股，但受限規定無法施行，希望放寬限制，但外界也都很理解此機制目的是保護股東權益，因此是否有可能在一定條件下有放寬可行性，金管會會再與券商公會研議處理。
證期局並補充，從掛牌日起算在3個月內、無法實施庫藏股的上市櫃公司，目前共有20家。
----------------------------------------------------------------------

[31] 使用牌照稅4月開徵　疫情影響者有5大紓困方案可用

▲使用牌照稅開徵。（圖／記者吳靜君攝）
記者吳靜君／台北報導
財政部今（26）日舉行使用牌照稅開徵記者會，財政部表示，今年要繳稅的車輛有766萬輛，較去年增加29萬輛、年增率3.93%；查定稅額有640億元，增加14億元、年增率2.17%。2020年使用牌照稅4月1日開徵，繳納期限到4月30日。
台南市政府財政稅務局長陳柏誠表示，政府收稅但不是「死要錢」，民眾益受疫情影響無法如期申報，或者無力如期繳稅有五大紓困方案可以運用。
一，因應新冠肺炎（COVID-19）疫情影響，交通部補貼遊覽車客運業、小客車租賃業使用牌照稅50%，以本次開徵為利，全國受惠車輛數有15萬輛，其中1.5萬輛

▲台灣菸酒。（圖／記者李毓康攝）
記者吳靜君／台北報導
受新冠肺炎疫情衝擊影響，民生消費經濟愈發吃緊，為和全民一起守護台灣經濟。台灣菸酒公司董事長丁彥哲（26）日宣佈，為響應行政院紓困作為，4月份起全國所屬3個啤酒廠，9個酒廠，共12家展售中心櫃位租金減免25%，即打75折，期減輕合作廠商經營上的租金壓力。
台灣菸酒公司在此次新冠肺炎疫情衝擊期間，不僅全力量產防疫酒精，為進一步守護台灣經濟，四月份起將針對轄下所屬各酒、啤酒廠共12家展售中心櫃位，推出租金減免25%的共體時艱方案。台酒表示，每個酒廠的攤商多寡不一，以埔里酒廠來說，每個月2500萬元的收入、大約有20到30家攤商、每家攤商每個月租金大約4~5萬元，租金減少四分之一可以減輕負擔。
丁彥哲表示，新冠肺炎（COVID-19）疫情持續升溫影響，民眾減少出門及消費，公司所屬各酒廠的展售中心營業額也受到衝擊，連帶使得承租櫃位54個合作商家，整體營收業績下滑。以全國人氣最高的埔里酒廠觀光工廠來說，今年2月份營業額比去年同期衰退達16%，合作廠商及商家營業額衰退高達23%，觀光來客數衰退3%。
丁彥哲表示，為與合作廠商共同渡過這段營運艱困時間，台酒公司推出共體時艱方案，即4月到6月共計三個月期間，針對轄下各酒、啤酒廠共12家展售中心的合作廠商，提供租金減免25%的優惠措施，減租期間將視疫情發展檢討調整，期能緩解合作廠商資金壓力，減輕疫情期間經營負擔。
▼台酒的展售與櫃位。（圖／台酒提供）
----------------------------------------------------------------------

[41] 2月全球手機銷量大跌14%　分析師：最糟的時刻還沒來

▲蘋果本月中旬宣布無限期暫時關閉大中華地區以外的實體門市。（圖／路透）
記者王曉敏／綜合報導
新冠肺炎疫情持續擴散對全球經濟造成衝擊，市場研究機構Counterpoint Research今（26）日表示，2月份全球智慧型手機銷量較去年同期下滑14%，隨著疫情持續惡化，智慧手機市場恐面臨進一步衰退。
因應疫情，中國大陸許多實體門市陸續關閉，以蘋果舉例，蘋果於2月初宣布關閉中國大陸的所有零售店面，於本月初陸續重啟營運後，又於本月中旬宣布無限期暫時關閉大中華地區以外的實體門市，最新消息稱最快於4月上旬重新開放。
雖然有分析師指出，蘋

新冠肺炎疫情爆發，對全球航空業和旅遊業可以說是首當其衝，不過在疫情和緩後，航太產業仍是長期成長的市場。
【文／劉瓊雯】
新冠肺炎（COVID-19）疫情爆發，甚至持續在全球擴大，在疫情影響的影響下，世界各國陸續宣布「鎖國」，紛紛頒布旅遊限縮和禁止令，禁止國際航線往來，全球航班紛紛取消、停飛，種種措施均導致乘客人數銳減，航空業需求大幅萎縮，對全球航空業和旅遊業可以說是首當其衝，航空公司陷入更嚴重的困境，導致全球各大航空類股股價紛紛向下急挫探底。
波音成最大受災戶
這一波最大的受災戶應該是美國的製造業巨頭波音，過去波音與歐洲空中巴士公司兩強長期較勁，爭奪市場龍頭地位，但在一八至一九年之間，短短的五個月內，波音因737 MAX客機分別在印尼和衣索比亞發生兩起致命空難，各國航管當局下令航空業者停飛737 MAX客機，波音就面臨飛機訂單減少與減產等危機，業績大受打擊，營收從一八年的一○一一億美元，下降到七六六億美元，每股盈餘從十六．○一美元至虧損三．四七美元；股價更是從四六四．○一美元高價跌到一○一．二五美元，大跌七七．三％，市值更是低至五七一．二八億美元；自從二○二○年新冠肺炎疫情爆發後，三月以來，波音在美股累計下跌超過六○％，三月十九日股價正式跌破一○○美元，與一九年同期相比跌了近八○％，截至三月二十日總市值只剩五三六．一億美元。對於不斷下挫的股價，波音曾要求當局及金融機構對該公司與供應鏈廠商們提供六○○億美元規模的資金援助，波音指出，如果沒有立法和適當的政策，美國的航空製造業可能會崩壞。
航太產業市場仍長期成長
澳洲智庫「亞太航空中心」（CAPA Centre for Aviation）出具的最新報告中也示警，飛機停飛導致各家航空公司的現金儲備正迅速減少，大部分航空公司將可能在五月底時破產，唯有政府與業界共同採取行動，才能避免這場災難發生。
做為美國最大的製造企業之一，光是在美國國內，波音的員工就超過萬名以上，此外，波音的供應商更超過一七○○○家，且波音還跨足軍機、飛彈以及太空科技等，這樣的大企業，美國總統川普在三月十七日的記者會上表示：「波音不救不行」，可見政府不可能對波音面臨的困境袖手旁觀。外資美林證券也認為，波音對美國經濟太重要，其生產的商用客機屬於雙寡占行業，需要長期大量資金的投入，是「太大到不能倒」的關鍵企業。
高盛集團（Goldman Sachs）在三月二十

▲英國保誠人壽宣布，取消現售10張醫療險保單的「法定傳染病30天等待期」。（圖／英國保誠人壽提供）
記者楊絡懸／台北報導
再添一家保險業者取消醫療保單等待期！英國保誠人壽26日宣布，除了率先推出新冠肺炎（COVID-19）「住院慰問金」之外，並放寬條件，取消現售10張醫療險保單的「法定傳染病30天等待期」，同時提供保戶緩繳保費的服務。這是第6家保險公司取消等待期，目前總計有67張醫療保單取消這項約定。
英國保誠人壽表示，即日起至7月底止，針對「e保庇健康保險」、「倍感安心終身醫療健康保險」等共計10張醫療險保單，取消法定傳染病30天等待期的約定。於此期間，保戶若因法定傳染病（包含COVID-19新型冠狀病毒）不幸經醫師確診需就醫，將可依保單條款約定獲得理賠，享有住院醫療的保障。
針對理賠及繳費等項目，英國保誠人壽也提供如快速理賠、理賠專線、理賠升級、保費緩繳作業等，受到疫情影響的保戶，可以填寫「災區保費緩繳申請書」，或致電向英國保誠人壽申請保費緩繳3個月。未來若疫情擴大，將統一依據政府所宣布之區域，主動給予保戶保費緩繳。同時，保戶若因新冠肺炎住進「負壓隔離病房」治療，將主動視為「加護病房」給付。
取消法定傳染病30天等待期的壽險業者，目前有國泰人壽、中國人壽、南山人壽、新光人壽及三商美邦人壽，其中，國泰人壽有10張現售住院醫療險保單、中國人壽9張、南山人壽有17張醫療險、新光人壽有15張住院醫療、手術醫療及實支實付醫療等、三商美邦人壽6張住院醫療保險，加上英國保誠人壽10張醫療險，共67張保單對新冠肺炎「現買現保障」，確診肺炎即理賠。
----------------------------------------------------------------------

[58] 股東會防疫指引祭2招應變　顧立雄估電子投票比例會大幅增加

▲金管會主委顧立雄。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
股東會旺季即將到來，外界擔憂上市櫃公司股東會人群聚集，恐成為防疫破口，金管會主委顧立雄今（26）日表示，請集保統計多少家股東會超過百人，研議相關配套措施，如更改場所、減少工作人員等，同時也推估「電子投票」比例一定會大幅增加。
指揮中心表示，室內超過100人以上、室外超過500人以上的集會活動建議停辦，立委關注股東會旺季即將到來，恐成為防疫破口。金管會主委顧立雄表

▲在市場對黃金需求激增之際，全球3大黃金精煉廠宣布因新冠肺炎疫情而停產，金價因而上漲。（圖／路透）
記者王曉敏／綜合報導
貴金屬價格近日隨著股市一同暴跌，不過在聯準會（Fed）宣布啟動無限制QE措施，及全球3大黃金精煉廠Valcambi、PAMP、Argor- Heraeus因疫情暫停生產的消息一出，金價出現強勁反彈，4月黃金期貨周二（24日）收盤大漲93.2美元或6%至每盎司1,660.80美元，單日漲額再寫1984年有紀錄以來最大，單日漲幅也創2009年以來最高。
近期隨著美股跳水，投資人將黃金變現以追加保證金的壓力也提高，金價於本月大起大落，部分投資人也透過出售黃金來購買美元等資產。不過在積極的刺激措施減少了資金壓力之後，投資人本周開始又重新進入市場。
但在市場對黃金需求激增之際，全球3大黃金精煉廠周一（23日）宣布因新冠肺炎疫情而停產至少一周，使金條、金幣等供應出現短缺，金價因而上漲。目前為止，黃金現貨價格本周已上漲近8%。
瑞士黄金安全公司（Swiss Gold Safe）董事Ludwig Karl表示，危機期間，政府已關閉了許多業務，使得人們越來越難以獲得實體黃金，「如果有人要購買黃金，我希望他們一切順利。大多數金銀交易商都關門了。」
IBV International Vaults黃金銷售總監Debra Thomson說：「整個供應鏈變得越來越緊張。我最後一次看到市場如此混亂是在911事件時。」他指出，該公司仍在銷售黃金，但不得不推遲交貨。
對那些想要接觸到實體黃金的人來說，難度已變得更大。新冠肺炎的大流行已使一些最受歡迎的金幣出現短缺，比如南非的克魯格金幣（Krugerrand）及加拿大的楓葉金幣（Maple Leaf）等。這意味著，人們現在在購買金幣時也已不再挑剔。Merrion Vaults的聯合創始人Seamus Fahy說：「兩個月前，買家可能會對他們想要的金幣精挑細選，在他們會買任何能到手的黃金。」
截至稍早，4月黃金期貨價格0.26%至1,630.1美元，黃金現貨則下跌0.57%至1,608.36美元。
----------------------------------------------------------------------

[69] 「室內100人限制」衝擊表演藝術！國表藝：能持續演出節目屈指可數

▲國表藝中心三場

▲工運大老賴萬枝性侵女秘書，二審仍判刑3年2月。（圖／翻攝全國金融業工會聯合總會官網）
記者吳銘峯／台北報導
工運大老賴萬枝2014年與女祕書赴南投參加勞工教育營，竟躲進女祕書房間中，壓在她身上以手指性侵，女祕書奮力反抗，賴萬枝竟說：「這件事情我有錯，但是你也有錯，因為你...長得很漂亮！」一審將賴男判刑3年2月。案經上訴，高等法院二審26日宣判，駁回上訴，維持刑期3年2月。
賴萬枝曾多次為勞工、女性等弱勢族群發聲，是台灣勞工運動領袖之一，還曾入選民進黨2012年不分區立委名單。而判決指出，賴萬枝2014年6月擔任「全國金融業工會聯合總會副理事長」時，與女祕書至南投縣明山森林會館參加2天1夜勞工教育活動，晚餐後幾位幹部一起歡唱KTV、飲酒作樂，喝到最後，賴萬枝與數名友人便跑到女秘書房間休息。
眾人陸續離開女祕書房間，但賴萬枝竟躲在房內，等女祕書回房休息，正準備放鬆入睡時，卻發現賴萬枝撲到自己身上，還啃咬她的乳頭，同時手還不停亂戳她的下體。女秘書放聲尖叫「你給我走，你給我滾！」使盡全力將賴男推向門外，但賴男竟反手關門，還淫笑說「這件事情，我有錯，但是妳也有錯，因為妳太優秀了，而且長得很漂亮！」女祕書奮力抵抗，賴萬枝見無法繼續，悻悻然離去。
女秘書事後報警提告，賴萬枝遭提起公訴。一審法院審理時，賴萬枝否認犯罪，辯護律師還說，被害人案發後沒有立刻報警，隔天還能與賴萬枝等人正常互動，不符常情。但法院認為，性侵被害人的反應會隨當時情況而定，所以女秘書在事後的活動中選擇保持沉默很合理；況且同行友人證實，女秘書的反應不像平常熱絡，還有人聽見賴萬枝對她道歉，並承諾賠償。因此一審認定賴萬枝有罪，判刑3年2月。
----------------------------------------------------------------------

[82] 油價連五降！專家估下周汽油零售價再降0.7元

▲國際油價走勢近期波動劇烈，專家預估下周油價可望連五降。（圖／記者林敬旻攝）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情延燒，衝擊全球旅運需求，專家預估下周國內油價可望再度調降，預估汽、柴油每公升零售價調降0.7元，台灣中油將在本周日中午公告實際零售價格，可望創「連五降」走勢。
依據中油目前油價估算，調整後的油價分別為92無鉛汽油每公升18.3元、95無鉛汽油每公升1

記者王致凱／綜合報導
美股開盤延續前日漲勢，道瓊工業指數漲幅迅速擴大至500點或2.47％，重新收復21000點大關。航空股延續漲勢，達美航空、美國航空、聯合航空漲幅均超過15％。標普500指數和那斯達克指數漲幅均擴大至超過1％。
受美國川普政府公布的2兆美元經濟振興方案激勵，美股道瓊周二大漲2100點或11％，創下1933年以來的單日最大漲幅。標普500指數同日大漲9.4％，創下2008年10月以來最大漲幅。
